In [ ]:
!pip install sentence-transformers faiss-cpu openai langchain chromadb --quiet


In [ ]:
from sentence_transformers import SentenceTransformer
from langchain.text_splitter import RecursiveCharacterTextSplitter
import faiss
import numpy as np
import openai


In [ ]:
# Replace this with your actual data
raw_text = """
Dr. Smith is a cardiologist with 10 years of experience...
Patients with diabetes should maintain a low glycemic index diet...
"""

text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
documents = text_splitter.create_documents([raw_text])
docs = [doc.page_content for doc in documents]


In [ ]:
model = SentenceTransformer('all-MiniLM-L6-v2')
doc_embeddings = model.encode(docs)


In [ ]:
dimension = len(doc_embeddings[0])
index = faiss.IndexFlatL2(dimension)
index.add(np.array(doc_embeddings))


In [ ]:
query = "What should a diabetic patient eat?"
query_embedding = model.encode([query])
_, indices = index.search(np.array(query_embedding), k=3)
relevant_chunks = [docs[i] for i in indices[0]]
print("Retrieved Chunks:\n", "\n---\n".join(relevant_chunks))


In [ ]:
openai.api_key = "YOUR_OPENAI_API_KEY"  # Replace with your actual key

context = "\n".join(relevant_chunks)
prompt = f"Context:\n{context}\n\nQuestion: {query}\nAnswer:"

response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=[{"role": "user", "content": prompt}]
)

print("LLM Response:\n", response['choices'][0]['message']['content'])
